In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

#### load Yedoma dataset 

In [69]:
gdf_pf = gpd.read_file('data/IPA_map_PF_Ground_ice/permaice.shp')

#### create dense fisnet for later reprocessing 

In [3]:
def create_fishnet(xmin, ymin, xmax, ymax, y_grid_size=1, x_grid_size=0.1):
    # Create a list of x and y coordinates for each cell in the fishnet
    y_coords = list(np.arange(int(ymin), int(ymax) + y_grid_size, y_grid_size))
    x_coords = list(np.arange(int(xmin), int(xmax) + x_grid_size, x_grid_size))

    # Create a list of polygons for each cell in the fishnet
    polygons = []
    for y in y_coords:
        for x in x_coords:
            polygons.append(Polygon([(x, y), (x + x_grid_size, y), (x + x_grid_size, y + y_grid_size), (x, y + y_grid_size)]))
    
    # Create a GeoDataFrame from the list of polygons
    fishnet = gpd.GeoDataFrame(geometry=polygons)
    
    # Set the coordinate reference system (CRS) to WGS84
    fishnet.crs = 'EPSG:4326'
    
    # Add the southern latitude coordinate as an attribute
    fishnet['latitude'] = fishnet.geometry.bounds['miny']
    
    return fishnet

In [70]:
cleaned = gdf_pf.dropna(subset='EXTENT')

In [15]:
xgrid=0.1
fishnet = create_fishnet(-180, 20, 180, 90, x_grid_size=xgrid, y_grid_size=1)

In [45]:
fishnet_EA = fishnet.to_crs('+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs')

In [46]:
#calculate intersection
#intersection = fishnet.overlay(cleaned, how='intersection')
intersection = fishnet_EA.overlay(cleaned, how='intersection')

/isipd/projects-noreplica/p_initze/anaconda3/envs/ML2/lib/python3.9/site-packages/geopandas/geodataframe.py:2352: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +ellps ...
Right CRS: EPSG:4326

  return geopandas.overlay(


#### fix

In [85]:
# remove n of 85°
fishnet_EA_filtered = fishnet_EA[(fishnet_EA['latitude'] < 85)]

In [86]:
# calculate intersection
intersection = fishnet_EA_filtered.overlay(cleaned, how='intersection')

/isipd/projects-noreplica/p_initze/anaconda3/envs/ML2/lib/python3.9/site-packages/geopandas/geodataframe.py:2352: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +ellps ...
Right CRS: PROJCS["Sphere_ARC_INFO_Lambert_Azimuthal_Equal_Ar ...

  return geopandas.overlay(


In [87]:
intersection['pf_area'] = intersection.area * 1e-6

In [88]:
intersection.to_file('intersected_PF_v2.gpkg')

In [89]:
grouped = intersection.groupby(['latitude', 'EXTENT'], as_index=False).sum()

In [94]:
grouped.to_csv('pfextent_area_km2_per_latitude_01grid.csv')
#yedoma_area_grouped.to_csv('yedoma_area_km2_per_latitude_001grid.csv')